In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import norm

# Define the mappings for renaming columns
column_mapping = {
    "Source IP": "Src IP",
    "Source Port": "Src Port",
    "Destination IP": "Dst IP",
    "Destination Port": "Dst Port",
    "Total Fwd Packets": "Total Fwd Packet",
    "Total Backward Packets": "Total Bwd packets",
    "Total Length of Fwd Packets": "Total Length of Fwd Packet",
    "Total Length of Bwd Packets": "Total Length of Bwd Packet",
    "Min Packet Length": "Packet Length Min",
    "Max Packet Length": "Packet Length Max",
    "Avg Fwd Segment Size": "Fwd Segment Size Avg",
    "Avg Bwd Segment Size": "Bwd Segment Size Avg",
    "CWE Flag Count": "CWR Flag Count",
    "Init_Win_bytes_forward": "FWD Init Win Bytes",
    "Init_Win_bytes_backward": "Bwd Init Win Bytes",
    "act_data_pkt_fwd": "Fwd Act Data Pkts",
    "min_seg_size_forward": "Fwd Seg Size Min"
}

# Get the list of CSV file paths
dspaths = []
for dirname, _, filenames in os.walk('/TFM/DATASET-1/v2/csv/'):
    for filename in filenames:
        if filename.endswith('.csv'):
            pds = os.path.join(dirname, filename)
            dspaths.append(pds)
            print(f"Found file: {pds}")

print(f"Total files found: {len(dspaths)}")

# Load the CSV files into a dictionary of DataFrames
data_frames = {}
for file in dspaths:
    file_name = os.path.basename(file)
    data_frames[file_name] = pd.read_csv(file)
    print(f"Loaded file: {file_name}, shape: {data_frames[file_name].shape}")

    # Rename the columns according to the mapping
    data_frames[file_name].rename(columns=column_mapping, inplace=True)

    # Update the 'Label' column based on the 'Traffic Subtype' and 'Label' values
    data_frames[file_name]['Label'] = data_frames[file_name].apply(
        lambda row: row['Traffic Subtype'] if row['Label'] == 'Malicious' else 'BENIGN' if row['Label'] == 'Benign' else row['Label'],
        axis=1
    )

    # Save the updated DataFrame back to CSV
    output_path = os.path.join('/TFM/DATASET-1/v2/updated_csv/', file_name)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    data_frames[file_name].to_csv(output_path, index=False)
    print(f"Updated file saved: {output_path}")

Found file: /TFM/DATASET-1/v2/csv/data.csv
Total files found: 1
Loaded file: data.csv, shape: (8656767, 86)
Updated file saved: /TFM/DATASET-1/v2/updated_csv/data.csv


In [20]:
import os
import pandas as pd

# Lista de columnas a eliminar
columns_to_remove = [
    "Flow ID", "Src IP", "Src Port", "Dst IP", "Dst Port", 
    "Timestamp", "Traffic Type", "Traffic Subtype", "CWR Flag Count"
]

# Obtener la lista de rutas de los archivos CSV
dspaths = []
for dirname, _, filenames in os.walk('/TFM/DATASET-1/v2/updated_csv/'):
    for filename in filenames:
        if filename.endswith('.csv'):
            pds = os.path.join(dirname, filename)
            dspaths.append(pds)
            print(f"Archivo encontrado: {pds}")

print(f"Total de archivos encontrados: {len(dspaths)}")

# Cargar los archivos CSV y eliminar las columnas especificadas
for file in dspaths:
    file_name = os.path.basename(file)
    print(f"Procesando archivo: {file_name}")
    
    # Leer el CSV en chunks para manejar archivos grandes
    chunk_size = 100000  # Ajusta este valor según la memoria disponible
    chunks = pd.read_csv(file, chunksize=chunk_size)
    
    # Procesar cada chunk
    processed_chunks = []
    for chunk in chunks:
        # Verificar si las columnas a eliminar existen en el DataFrame
        existing_columns_to_remove = [col for col in columns_to_remove if col in chunk.columns]
        
        # Eliminar las columnas no deseadas
        chunk = chunk.drop(columns=existing_columns_to_remove)
        
        # Convertir todas las columnas a tipo string para evitar problemas de tipo de datos
        chunk = chunk.astype(str)
        
        # Reemplazar posibles valores no válidos
        chunk = chunk.replace({'nan': '', 'inf': '', '-inf': ''})
        
        processed_chunks.append(chunk)
    
    # Concatenar todos los chunks procesados
    df = pd.concat(processed_chunks, ignore_index=True)
    
    # Guardar el DataFrame actualizado
    output_path = os.path.join('/TFM/DATASET-1/v2/final_csv/', file_name)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False, encoding='utf-8', quoting=1)
    print(f"Archivo actualizado guardado: {output_path}, shape: {df.shape}")

print("Procesamiento completado.")

Archivo encontrado: /TFM/DATASET-1/v2/updated_csv/data.csv
Total de archivos encontrados: 1
Procesando archivo: data.csv
Archivo actualizado guardado: /TFM/DATASET-1/v2/final_csv/data.csv, shape: (8656767, 77)
Procesamiento completado.


#### Codigo momentaneo para calcular la longitud maxima del dataframe y los datos por etiquetas

In [17]:
"""
# Crear un DataFrame con todas las muestras benignas

# Initialize dictionaries for benign and malignant samples
benign_samples = pd.DataFrame()
malign_samples_by_label = {
    'Bruteforce': pd.DataFrame(),
    'DoS': pd.DataFrame(),
    'Information Gathering': pd.DataFrame(),
    'Mirai': pd.DataFrame()
}

# Process each DataFrame
for file, df in data_frames.items():
    print(f"Processing file: {file}")
    if file in ['processed_Audio.csv', 'processed_Text.csv', 'processed_Video.csv', 'processed_Background.csv']:
        # These files are considered BENIGN
        benign_samples = pd.concat([benign_samples, df])
        print(f"Added {len(df)} rows to BENIGN")
    else:
        # Determine the attack type based on the file name
        if 'Bruteforce' in file:
            attack_type = 'Bruteforce'
        elif 'DoS' in file:
            attack_type = 'DoS'
        elif 'Information_Gathering' in file:
            attack_type = 'Information Gathering'
        elif 'Mirai' in file:
            attack_type = 'Mirai'
        else:
            print(f"Unknown file type: {file}")
            continue
        
        # Add samples to the corresponding attack type
        malign_samples_by_label[attack_type] = pd.concat([malign_samples_by_label[attack_type], df])
        print(f"Added {len(df)} rows to {attack_type}")
        
# Display the number of benign samples
print(f"Label: BENIGN, Number of samples: {len(benign_samples)}")

# Display the number of malignant samples per label
for label, samples in malign_samples_by_label.items():
    malign_count = len(samples)
    print(f"Label: {label}, Number of malignant samples: {malign_count}")
"""

Processing file: data.csv
Unknown file type: data.csv
Label: BENIGN, Number of samples: 0
Label: Bruteforce, Number of malignant samples: 0
Label: DoS, Number of malignant samples: 0
Label: Information Gathering, Number of malignant samples: 0
Label: Mirai, Number of malignant samples: 0


#### Realizamos la division de los datos en funcion de la distribuccion Gaussiana

In [29]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Gaussian distribution parameters
mu = 0.5
sigma = 0.2

# Initialize dictionary for reduced malignant samples by label
malign_reduced_samples_by_label = {}

# Check if malign_samples_by_label is empty or has no non-zero categories
if not malign_samples_by_label or all(len(samples) == 0 for samples in malign_samples_by_label.values()):
    print("No malignant samples found. Skipping malignant sample processing.")
    min_samples = 0
else:
    # Find the minimum number of samples among non-zero categories
    min_samples = min(len(samples) for label, samples in malign_samples_by_label.items() if len(samples) > 0)
    print(f"Target number of samples per attack type: {min_samples}")

    # Process each DataFrame
    for label, samples in malign_samples_by_label.items():
        if len(samples) == 0:
            print(f"Skipping {label} due to zero samples")
            continue
        
        # Calculate the probability density function of each sample
        samples['prob'] = norm.pdf(np.linspace(0, 1, len(samples)), mu, sigma)
        
        # Reduce samples to min_samples using the probability weights
        if len(samples) > min_samples:
            malign_reduced_samples_by_label[label] = samples.sample(n=min_samples, weights='prob', random_state=42)
        else:
            # If we have fewer samples than min_samples, use all available samples
            malign_reduced_samples_by_label[label] = samples
        
        print(f"Reduced {label} to {len(malign_reduced_samples_by_label[label])} samples")

    # Concatenate all reduced malignant samples into one DataFrame
    malign_reduced_data = pd.concat(malign_reduced_samples_by_label.values())

    # Drop the 'prob' column
    malign_reduced_data = malign_reduced_data.drop('prob', axis=1)

# Process benign samples
if 'benign_samples' in locals() and not benign_samples.empty:
    benign_data = benign_samples

    # Reduce benign samples to match the number of samples per attack type
    if min_samples > 0 and len(benign_data) > min_samples:
        benign_data = benign_data.sample(n=min_samples, random_state=42)
    print(f"Reduced BENIGN to {len(benign_data)} samples")

    # Concatenate all benign and reduced malignant samples into one DataFrame
    all_data = pd.concat([benign_data, malign_reduced_data]) if 'malign_reduced_data' in locals() else benign_data
else:
    print("No benign samples found.")
    all_data = malign_reduced_data if 'malign_reduced_data' in locals() else pd.DataFrame()

print("\nFinal sample counts:")
if 'benign_data' in locals():
    print(f"BENIGN: {len(benign_data)}")
for label, samples in malign_reduced_samples_by_label.items():
    print(f"{label}: {len(samples)}")

print(f"\nTotal samples in all_data: {len(all_data)}")

No malignant samples found. Skipping malignant sample processing.
No benign samples found.

Final sample counts:
BENIGN: 0

Total samples in all_data: 0


In [ ]:


from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
train_data, test_val_data = train_test_split(all_data, test_size=0.3, random_state=42)

# Dividir el conjunto de prueba en conjuntos de prueba y validación (50% prueba, 50% validación)
#test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=42)

# Guardar los conjuntos de entrenamiento en archivos CSV
train_data.to_csv('train.csv', index=False)

# Guardar los conjuntos de prueba en archivos CSV
test_val_data.to_csv('test.csv', index=False)

# Guardar los conjuntos de validación en archivos CSV
#val_data.to_csv('validation.csv', index=False)

In [ ]:
"""
# Crear un DataFrame con todas las muestras benignas
#benign_samples = pd.concat([df for df in data_frames.values() if df.iloc[:, 77].eq('BENIGN').any()])
benign_samples = pd.concat([df[df.iloc[:, 77].eq('BENIGN')] for df in data_frames.values()])
##benign_samples = benign_samples.sample(n=10000, random_state=42)
print(f"Length: {len(benign_samples)}")
# Reducir las muestras malignas de cada fichero utilizando la función gaussiana para seleccionar las muestras
mu = 0.5  # Media de la distribución gaussiana
sigma = 0.2  # Desviación estándar de la distribución gaussiana

malign_samples_reduced = {}
for file, df in data_frames.items():
    # Obtener solo las muestras malignas (sin etiqueta BENIGN)
    malign_samples = df[df.iloc[:, 77] != 'BENIGN']

    # Calcular las probabilidades gaussianas
    #malign_samples['prob'] = norm.pdf(np.linspace(0, 1, len(malign_samples)), mu, sigma)
    ##malign_samples.loc[:, 'prob'] = norm.pdf(np.linspace(0, 1, len(malign_samples)), mu, sigma)

    # Reducir las muestras malignas a 10000 por fichero
    ##malign_samples_reduced[file] = malign_samples.sample(n=10000, weights='prob', random_state=42)

"""

'\n# Crear un DataFrame con todas las muestras benignas\n#benign_samples = pd.concat([df for df in data_frames.values() if df.iloc[:, 77].eq(\'BENIGN\').any()])\nbenign_samples = pd.concat([df[df.iloc[:, 77].eq(\'BENIGN\')] for df in data_frames.values()])\n##benign_samples = benign_samples.sample(n=10000, random_state=42)\nprint(f"Length: {len(benign_samples)}")\n# Reducir las muestras malignas de cada fichero utilizando la función gaussiana para seleccionar las muestras\nmu = 0.5  # Media de la distribución gaussiana\nsigma = 0.2  # Desviación estándar de la distribución gaussiana\n\nmalign_samples_reduced = {}\nfor file, df in data_frames.items():\n    # Obtener solo las muestras malignas (sin etiqueta BENIGN)\n    malign_samples = df[df.iloc[:, 77] != \'BENIGN\']\n\n    # Calcular las probabilidades gaussianas\n    #malign_samples[\'prob\'] = norm.pdf(np.linspace(0, 1, len(malign_samples)), mu, sigma)\n    ##malign_samples.loc[:, \'prob\'] = norm.pdf(np.linspace(0, 1, len(malign_sa

Creamos los ficheros con datos malignos todos juntos y benigno. Lo cramos una sola vez

In [ ]:
"""
# Concatenar todos los DataFrames con datos benignos en un solo DataFrame
benign_data = pd.concat([benign_samples])

# Concatenar todos los DataFrames con datos malignos reducidos en un solo DataFrame
malign_reduced_data = pd.concat(malign_samples_reduced)

# Eliminar la columna "prob" del DataFrame malign_reduced_data
malign_reduced_data = malign_reduced_data.drop('prob', axis=1)

# Guardar los datos benignos en un archivo CSV
benign_data.to_csv('benign_data.csv', index=False)

# Guardar los datos malignos reducidos en un archivo CSV sin la columna "prob"
malign_reduced_data.to_csv('malign_reduced_data.csv', index=False)
"""

'\n# Concatenar todos los DataFrames con datos benignos en un solo DataFrame\nbenign_data = pd.concat([benign_samples])\n\n# Concatenar todos los DataFrames con datos malignos reducidos en un solo DataFrame\nmalign_reduced_data = pd.concat(malign_samples_reduced)\n\n# Eliminar la columna "prob" del DataFrame malign_reduced_data\nmalign_reduced_data = malign_reduced_data.drop(\'prob\', axis=1)\n\n# Guardar los datos benignos en un archivo CSV\nbenign_data.to_csv(\'benign_data.csv\', index=False)\n\n# Guardar los datos malignos reducidos en un archivo CSV sin la columna "prob"\nmalign_reduced_data.to_csv(\'malign_reduced_data.csv\', index=False)\n'

División entre Train, Test y Validación

In [ ]:
"""
from sklearn.model_selection import train_test_split

# Concatenar los datos benignos y malignos en un solo DataFrame
all_data = pd.concat([benign_data, malign_reduced_data])

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
all_train, all_test = train_test_split(all_data, test_size=0.3, random_state=42)

# Dividir el conjunto de prueba en conjuntos de prueba y validación (20% prueba, 10% validación)
all_test, all_val = train_test_split(all_test, test_size=0.5, random_state=42)

# Separar los datos benignos y malignos en los conjuntos correspondientes
benign_train = all_train[all_train.iloc[:, 77] == 'BENIGN']
malign_train = all_train[all_train.iloc[:, 77] != 'BENIGN']
benign_test = all_test[all_test.iloc[:, 77] == 'BENIGN']
malign_test = all_test[all_test.iloc[:, 77] != 'BENIGN']
benign_val = all_val[all_val.iloc[:, 77] == 'BENIGN']
malign_val = all_val[all_val.iloc[:, 77] != 'BENIGN']

# Guardar los conjuntos de entrenamiento en archivos CSV
pd.concat([benign_train, malign_train]).to_csv('train.csv', index=False)

# Guardar los conjuntos de prueba en archivos CSV
pd.concat([benign_test, malign_test]).to_csv('test.csv', index=False)

# Guardar los conjuntos de validación en archivos CSV
pd.concat([benign_val, malign_val]).to_csv('validation.csv', index=False)

"""

"\nfrom sklearn.model_selection import train_test_split\n\n# Concatenar los datos benignos y malignos en un solo DataFrame\nall_data = pd.concat([benign_data, malign_reduced_data])\n\n# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)\nall_train, all_test = train_test_split(all_data, test_size=0.3, random_state=42)\n\n# Dividir el conjunto de prueba en conjuntos de prueba y validación (20% prueba, 10% validación)\nall_test, all_val = train_test_split(all_test, test_size=0.5, random_state=42)\n\n# Separar los datos benignos y malignos en los conjuntos correspondientes\nbenign_train = all_train[all_train.iloc[:, 77] == 'BENIGN']\nmalign_train = all_train[all_train.iloc[:, 77] != 'BENIGN']\nbenign_test = all_test[all_test.iloc[:, 77] == 'BENIGN']\nmalign_test = all_test[all_test.iloc[:, 77] != 'BENIGN']\nbenign_val = all_val[all_val.iloc[:, 77] == 'BENIGN']\nmalign_val = all_val[all_val.iloc[:, 77] != 'BENIGN']\n\n# Guardar los conjuntos de entrenam